First we need to download the Spark Connector for Neo4j from its Github repository

In [ ]:
import wget, os
url = 'https://github.com/neo4j-contrib/neo4j-spark-connector/releases/download/5.0.0/neo4j-connector-apache-spark_2.12-5.0.0_for_spark_3.jar'

# Getting the name of the Spark Connector from its URL
filename = url.split("/")[-1]

# Download it only if it's not present
if filename not in os.listdir():
    print("Downlading Spark Connector")
    filename = wget.download(url)

Let's test the connection of our cluster to Neo4j

In [ ]:
from pyspark.sql import SparkSession
import os

# Providing the downloaded jar to the pyspark-shell
os.environ[
    'PYSPARK_SUBMIT_ARGS'] = f'--jars {filename} pyspark-shell'

#Ip address of the Neo4j database, you need also to provide its port number
# If you want to connect to your local Neo4j instance in MacOS and Windows
# please use this instead of localhost -> host.docker.internal
NEO4J_IP_ADDRESS = "ip_address_of_neo4j:bolt_port_number(usually is 7687)"

# Building our SparkSession
spark = SparkSession.\
        builder.\
        appName("pyspark-notebook").\
        master("spark://spark-master:7077").\
        config("spark.executor.memory", "512m").\
        getOrCreate()

# Reading all the nodes from Neo4j
x = spark.read.format("org.neo4j.spark.DataSource") \
        .option("url", f"bolt://{NEO4J_IP_ADDRESS}") \
        .option("authentication.basic.username", "neo4j") \
        .option("authentication.basic.password", "test") \
        .option("query", "MATCH (n) RETURN n LIMIT 10") \
        .load()

# Showing the result of the read
x.show()

In [ ]:
# Close the SparkSession
spark.stop()